# 第2章 索引

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv',index_col='ID')
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


## 一、单级索引
### 1. loc方法、iloc方法、[]操作符
#### 最常用的索引方法可能就是这三类，其中iloc表示位置索引，loc表示标签索引，[]也具有很大的便利性，各有特点
#### （a）loc方法
#### ① 单行索引：

In [2]:
df.loc[1103]

School          S_1
Class           C_1
Gender            M
Address    street_2
Height          186
Weight           82
Math           87.2
Physics          B+
Name: 1103, dtype: object

#### ② 多行索引：

In [4]:
df.loc[1304:2103].head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1304,S_1,C_3,M,street_2,195,70,85.2,A
1305,S_1,C_3,F,street_5,187,69,61.7,B-
2101,S_2,C_1,M,street_7,174,84,83.3,C
2102,S_2,C_1,F,street_6,161,61,50.6,B+
2103,S_2,C_1,M,street_4,157,61,52.5,B-


#### ③ 单列索引：

In [6]:
df.loc[:,'Height'].head()

ID
1101    173
1102    192
1103    186
1104    167
1105    159
Name: Height, dtype: int64

#### ④ 多列索引：

In [8]:
df.loc[:,'Height':'Math'].head()

,Height,Weight,Math
ID,,,
1101,173,63,34.0
1102,192,73,32.5
1103,186,82,87.2
1104,167,81,80.4
1105,159,64,84.8


#### ⑤ 联合索引：

In [9]:
df.loc[1102:2401:3,'Height':'Math'].head()

,Height,Weight,Math
ID,,,
1102,192,73,32.5
1105,159,64,84.8
1203,160,53,58.8
1301,161,68,31.5
1304,195,70,85.2


#### ⑥ 函数式索引：

In [10]:
df.loc[lambda x:x['Gender']=='M'].head()
#loc中使用的函数，传入参数就是前面的df

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1203,S_1,C_2,M,street_6,160,53,58.8,A+
1301,S_1,C_3,M,street_4,161,68,31.5,B+


In [11]:
#这里的例子表示，loc中能够传入函数，并且函数的输入值是整张表，输出为标量、切片、合法列表（元素出现在索引中）、合法索引
def f(x):
    return [1101,1103]
df.loc[f]

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1103,S_1,C_1,M,street_2,186,82,87.2,B+


#### ⑦ 布尔索引（将重点在第2节介绍）

In [12]:
df.loc[df['Address'].isin(['street_7','street_4'])].head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1105,S_1,C_1,F,street_4,159,64,84.8,B+
1202,S_1,C_2,F,street_4,176,94,63.5,B-
1301,S_1,C_3,M,street_4,161,68,31.5,B+
1303,S_1,C_3,M,street_7,188,82,49.7,B
2101,S_2,C_1,M,street_7,174,84,83.3,C


### 3. 快速标量索引
#### 当只需要取一个元素时，at和iat方法能够提供更快的实现：

In [42]:
display(df.at[1101,'School'])
display(df.loc[1101,'School'])
display(df.iat[0,0])
display(df.iloc[0,0])
#可尝试去掉注释对比时间
#%timeit df.at[1101,'School']
#%timeit df.loc[1101,'School']
#%timeit df.iat[0,0]
#%timeit df.iloc[0,0]

'S_1'

'S_1'

'S_1'

'S_1'

### 4. 区间索引
#### 此处介绍并不是说只能在单级索引中使用区间索引，只是作为一种特殊类型的索引方式，在此处先行介绍
#### （a）利用interval_range方法

In [43]:
pd.interval_range(start=0,end=5)
#closed参数可选'left''right''both''neither'，默认左开右闭

IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]],
              closed='right',
              dtype='interval[int64]')

In [44]:
pd.interval_range(start=0,periods=8,freq=5)
#periods参数控制区间个数，freq控制步长

IntervalIndex([(0, 5], (5, 10], (10, 15], (15, 20], (20, 25], (25, 30], (30, 35], (35, 40]],
              closed='right',
              dtype='interval[int64]')

#### （b）利用cut将数值列转为区间为元素的分类变量，例如统计数学成绩的区间情况：

In [45]:
math_interval = pd.cut(df['Math'],bins=[0,40,60,80,100])
#注意，如果没有类型转换，此时并不是区间类型，而是category类型
math_interval.head()

ID
1101      (0, 40]
1102      (0, 40]
1103    (80, 100]
1104    (80, 100]
1105    (80, 100]
Name: Math, dtype: category
Categories (4, interval[int64]): [(0, 40] < (40, 60] < (60, 80] < (80, 100]]

## 三、索引设定
### 1. index_col参数
#### index_col是read_csv中的一个参数，而不是某一个方法：

In [71]:
pd.read_csv('data/table.csv',index_col=['Address','School']).head()

Class    ID Gender  Height  Weight  Math Physics
Address  School                                                 
street_1 S_1      C_1  1101      M     173      63  34.0      A+
street_2 S_1      C_1  1102      F     192      73  32.5      B+
         S_1      C_1  1103      M     186      82  87.2      B+
         S_1      C_1  1104      F     167      81  80.4      B-
street_4 S_1      C_1  1105      F     159      64  84.8      B+

### 2. reindex和reindex_like
#### reindex是指重新索引，它的重要特性在于索引对齐，很多时候用于重新排序

In [72]:
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [73]:
df.reindex(index=[1101,1203,1206,2402])

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173.0,63.0,34.0,A+
1203,S_1,C_2,M,street_6,160.0,53.0,58.8,A+
1206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2402,S_2,C_4,M,street_7,166.0,82.0,48.7,B


### 3. set_index和reset_index
#### 先介绍set_index：从字面意思看，就是将某些列作为索引

#### 使用表内列作为索引：

In [79]:
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [80]:
df.set_index('Class').head()

,School,Gender,Address,Height,Weight,Math,Physics
Class,,,,,,,
C_1,S_1,M,street_1,173,63,34.0,A+
C_1,S_1,F,street_2,192,73,32.5,B+
C_1,S_1,M,street_2,186,82,87.2,B+
C_1,S_1,F,street_2,167,81,80.4,B-
C_1,S_1,F,street_4,159,64,84.8,B+


#### 下面介绍reset_index方法，它的主要功能是将索引重置
#### 默认状态直接恢复到自然数索引：

In [84]:
df.reset_index().head()

,ID,School,Class,Gender,Address,Height,Weight,Math,Physics
0,1101,S_1,C_1,M,street_1,173,63,34.0,A+
1,1102,S_1,C_1,F,street_2,192,73,32.5,B+
2,1103,S_1,C_1,M,street_2,186,82,87.2,B+
3,1104,S_1,C_1,F,street_2,167,81,80.4,B-
4,1105,S_1,C_1,F,street_4,159,64,84.8,B+


### 4.  rename

#### rename方法用于修改列或者行索引标签，而不是索引名：

In [90]:
df_temp.rename(index={'A':'T'},columns={'e':'changed_e'}).head()

Big                 D                             E                      \
Small               d changed_e         f         d changed_e         f   
Upper Lower                                                               
T     a      0.952394  0.225359  0.088530  0.344681  0.803563  0.957546   
      b      0.597974  0.170867  0.713686  0.045497  0.013743  0.609154   
      c      0.827619  0.999765  0.431732  0.060878  0.402561  0.183220   
B     a      0.820653  0.562288  0.387569  0.310252  0.876846  0.525805   
      b      0.610984  0.052675  0.056680  0.815168  0.933346  0.712698   

Big                 F                      
Small               d changed_e         f  
Upper Lower                                
T     a      0.649799  0.644266  0.533074  
      b      0.642097  0.520484  0.113203  
      c      0.567167  0.822338  0.383110  
B     a      0.786711  0.638720  0.450830  
      b      0.042167  0.459474  0.934926

## 四、常用索引型函数
### 1. where函数
#### 当对条件为False的单元进行填充：

In [91]:
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


In [92]:
df.where(df['Gender']=='M').head()
#不满足条件的行全部被设置为NaN

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173.0,63.0,34.0,A+
1102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103,S_1,C_1,M,street_2,186.0,82.0,87.2,B+
1104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 第一个参数为布尔条件，第二个参数为填充值：

In [94]:
df.where(df['Gender']=='M',np.random.rand(df.shape[0],df.shape[1])).head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173.000000,63.000000,34.000000,A+
1102,0.880363,0.377656,0.441071,0.192081,0.596748,0.693048,0.809448,0.41425
1103,S_1,C_1,M,street_2,186.000000,82.000000,87.200000,B+
1104,0.432909,0.660837,0.90067,0.93032,0.099089,0.449954,0.426169,0.082895
1105,0.540073,0.68175,0.262715,0.336918,0.714834,0.642204,0.956307,0.465849


### 2. mask函数
#### mask函数与where功能上相反，其余完全一致，即对条件为True的单元进行填充

In [95]:
df.mask(df['Gender']=='M').dropna().head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1102,S_1,C_1,F,street_2,192.0,73.0,32.5,B+
1104,S_1,C_1,F,street_2,167.0,81.0,80.4,B-
1105,S_1,C_1,F,street_4,159.0,64.0,84.8,B+
1202,S_1,C_2,F,street_4,176.0,94.0,63.5,B-
1204,S_1,C_2,F,street_5,162.0,63.0,33.8,B


In [96]:
df.mask(df['Gender']=='M',np.random.rand(df.shape[0],df.shape[1])).head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,0.273904,0.460798,0.446225,0.633699,0.895552,0.296590,0.002112,0.222349
1102,S_1,C_1,F,street_2,192.000000,73.000000,32.500000,B+
1103,0.266646,0.567703,0.0981018,0.625369,0.876915,0.405576,0.508490,0.203879
1104,S_1,C_1,F,street_2,167.000000,81.000000,80.400000,B-
1105,S_1,C_1,F,street_4,159.000000,64.000000,84.800000,B+


### 3. query函数

In [97]:
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


#### query函数中的布尔表达式中，下面的符号都是合法的：行列索引名、字符串、and/not/or/&/|/~/not in/in/==/!=、四则运算符

In [98]:
df.query('(Address in ["street_6","street_7"])&(Weight>(70+10))&(ID in [1303,2304,2402])')

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1303,S_1,C_3,M,street_7,188,82,49.7,B
2304,S_2,C_3,F,street_6,164,81,95.5,A-
2402,S_2,C_4,M,street_7,166,82,48.7,B


## 五、重复元素处理
### 1. duplicated方法
#### 该方法返回了是否重复的布尔列表

In [99]:
df.duplicated('Class').head()

ID
1101    False
1102     True
1103     True
1104     True
1105     True
dtype: bool

#### 可选参数keep默认为first，即首次出现设为不重复，若为last，则最后一次设为不重复，若为False，则所有重复项为True

In [100]:
df.duplicated('Class',keep='last').tail()

ID
2401     True
2402     True
2403     True
2404     True
2405    False
dtype: bool

In [101]:
df.duplicated('Class',keep=False).head()

ID
1101    True
1102    True
1103    True
1104    True
1105    True
dtype: bool

### 2. drop_duplicates方法
#### 从名字上看出为剔除重复项，这在后面章节中的分组操作中可能是有用的，例如需要保留每组的第一个值：

In [102]:
df.drop_duplicates('Class')

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1301,S_1,C_3,M,street_4,161,68,31.5,B+
2401,S_2,C_4,F,street_2,192,62,45.3,A


#### 参数与duplicate函数类似：

In [103]:
df.drop_duplicates('Class',keep='last')

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2105,S_2,C_1,M,street_4,170,81,34.2,A
2205,S_2,C_2,F,street_7,183,76,85.4,B
2305,S_2,C_3,M,street_4,187,73,48.9,B
2405,S_2,C_4,F,street_6,193,54,47.6,B


#### 在传入多列时等价于将多列共同视作一个多级索引，比较重复项：

In [104]:
df.drop_duplicates(['School','Class'])

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1301,S_1,C_3,M,street_4,161,68,31.5,B+
2101,S_2,C_1,M,street_7,174,84,83.3,C
2201,S_2,C_2,M,street_5,193,100,39.1,B
2301,S_2,C_3,F,street_4,157,78,72.3,B+
2401,S_2,C_4,F,street_2,192,62,45.3,A


## 六、抽样函数
#### 这里的抽样函数指的就是sample函数
#### （a）n为样本量

In [105]:
df.sample(n=5)

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1201,S_1,C_2,M,street_5,188,68,97.0,A-
1202,S_1,C_2,F,street_4,176,94,63.5,B-
1302,S_1,C_3,F,street_1,175,57,87.7,A-
1205,S_1,C_2,F,street_6,167,63,68.4,B-
2202,S_2,C_2,F,street_7,194,77,68.5,B+


#### （b）frac为抽样比

In [106]:
df.sample(frac=0.05)

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2303,S_2,C_3,F,street_7,190,99,65.9,C
1301,S_1,C_3,M,street_4,161,68,31.5,B+


#### （c）replace为是否放回

In [107]:
df.sample(n=df.shape[0],replace=True).head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2205,S_2,C_2,F,street_7,183,76,85.4,B
1202,S_1,C_2,F,street_4,176,94,63.5,B-
1301,S_1,C_3,M,street_4,161,68,31.5,B+
1303,S_1,C_3,M,street_7,188,82,49.7,B
2402,S_2,C_4,M,street_7,166,82,48.7,B


In [108]:
df.sample(n=35,replace=True).index.is_unique

False

#### （d）axis为抽样维度，默认为0，即抽行

In [109]:
df.sample(n=3,axis=1).head()

,Address,Height,Math
ID,,,
1101,street_1,173,34.0
1102,street_2,192,32.5
1103,street_2,186,87.2
1104,street_2,167,80.4
1105,street_4,159,84.8


#### （e）weights为样本权重，自动归一化

In [110]:
df.sample(n=3,weights=np.random.rand(df.shape[0])).head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
2402,S_2,C_4,M,street_7,166,82,48.7,B
1201,S_1,C_2,M,street_5,188,68,97.0,A-


In [111]:
#以某一列为权重，这在抽样理论中很常见
#抽到的概率与Math数值成正比
df.sample(n=3,weights=df['Math']).head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
2404,S_2,C_4,F,street_2,160,84,67.7,B
2304,S_2,C_3,F,street_6,164,81,95.5,A-
1101,S_1,C_1,M,street_1,173,63,34.0,A+
